In [48]:
from sklearn.ensemble import RandomForestClassifier
import my_functions as myfns
import warnings
warnings.simplefilter(action='ignore', category=FutureWarning)
from sklearn.metrics import accuracy_score, precision_score
import pandas as pd
import numpy as np
import os

%load_ext autoreload
%autoreload 2

The autoreload extension is already loaded. To reload it, use:
  %reload_ext autoreload


In [43]:
datasets = ['test','train','validate']
os.chdir('C:/Users/Bernard/OneDrive/Documents/Viz/Ass02/CNN_3/kvasir-dataset-v2/test')
labels = os.listdir('.')
os.chdir('../')
path = os.getcwd()
features = ['JCD', 'Tamura', 'Colorlayout','EdgeHistogram','AutoColorCorrelogram','PHOG']
feature_lengths = [168, 18, 33, 80, 256, 630]
cols = ['filename','dataset','label']
for i in range(len(features)):
    j = 1
    while j <= feature_lengths[i]:
        cols.append(features[i] + '_' + str(j))
        j += 1
        print(cols)
df = pd.DataFrame(columns=cols)
df

IOPub data rate exceeded.
The notebook server will temporarily stop sending output
to the client in order to avoid crashing it.
To change this limit, set the config variable
`--NotebookApp.iopub_data_rate_limit`.

Current values:
NotebookApp.iopub_data_rate_limit=1000000.0 (bytes/sec)
NotebookApp.rate_limit_window=3.0 (secs)



,filename,dataset,label,JCD_1,JCD_2,JCD_3,JCD_4,JCD_5,JCD_6,JCD_7,...,PHOG_621,PHOG_622,PHOG_623,PHOG_624,PHOG_625,PHOG_626,PHOG_627,PHOG_628,PHOG_629,PHOG_630


In [44]:
os.chdir('C:/Users/Bernard/OneDrive/Documents/Viz/Ass02/CNN_3/kvasir-dataset-v2')
path = os.getcwd()
print('Processing Global Feature Files...')
for dataset in datasets:
    print(dataset.upper() + ' files :')
    for label in labels:
        path_tmp = path + '/' + dataset + '/' + label + '/'
        os.chdir(path_tmp)
        files = [file for file in os.listdir('.') if file.endswith(".features")]
        print(str(len(files)) + ' ' + label + ' files...')
        df = myfns.get_features(files, dataset, label, df)

Processing Global Feature Files...
TEST files :
150 dyed-lifted-polyps files...
150 dyed-resection-margins files...
150 esophagitis files...
150 normal-cecum files...
150 normal-pylorus files...
150 normal-z-line files...
150 polyps files...
150 ulcerative-colitis files...
TRAIN files :
700 dyed-lifted-polyps files...
700 dyed-resection-margins files...
700 esophagitis files...
700 normal-cecum files...
700 normal-pylorus files...
700 normal-z-line files...
700 polyps files...
700 ulcerative-colitis files...
VALIDATE files :
150 dyed-lifted-polyps files...
150 dyed-resection-margins files...
150 esophagitis files...
150 normal-cecum files...
150 normal-pylorus files...
150 normal-z-line files...
150 polyps files...
150 ulcerative-colitis files...


In [59]:
# 1. ALL features for each dataset
# TRAIN
df_train = df.loc[df['dataset']=='train'].iloc[:,2:] # train dataset + drop 1st 2 cols
y_train = df_train['label'] # 1st (label) col only
X_train = df_train.iloc[:,1:] # drop 1st (label) col
# TEST
df_test = df.loc[df['dataset']=='test'].iloc[:,2:] # test dataset + drop 1st 2 cols
y_test = df_test['label'] # 1st (label) col only
X_test = df_test.iloc[:,1:] # drop 1st (label) col
X_test


,JCD_1,JCD_2,JCD_3,JCD_4,JCD_5,JCD_6,JCD_7,JCD_8,JCD_9,JCD_10,...,PHOG_621,PHOG_622,PHOG_623,PHOG_624,PHOG_625,PHOG_626,PHOG_627,PHOG_628,PHOG_629,PHOG_630
0,0.0,1.5,4.5,0.0,0.0,0.0,2.5,1.5,3.5,0.0,...,0.0,6.0,0.0,0.0,0.0,0.0,2.0,3.0,0.0,0.0
1,3.0,3.5,6.0,0.0,0.0,0.0,1.0,0.0,0.5,0.5,...,8.0,4.0,1.0,3.0,3.0,1.0,2.0,14.0,2.0,0.0
2,1.0,1.5,5.5,0.0,0.0,0.0,3.0,3.0,5.0,1.0,...,9.0,5.0,3.0,8.0,6.0,4.0,1.0,4.0,3.0,8.0
3,0.5,3.5,6.0,0.0,0.0,0.0,3.0,1.5,3.5,0.0,...,9.0,8.0,8.0,9.0,10.0,11.0,6.0,5.0,7.0,10.0
4,0.0,0.0,5.5,0.0,0.0,0.0,3.5,4.0,5.0,0.0,...,6.0,2.0,2.0,4.0,5.0,2.0,1.0,2.0,2.0,4.0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
1195,0.0,0.0,5.0,0.0,0.0,0.0,4.0,6.5,2.0,0.0,...,12.0,14.0,11.0,1.0,0.0,0.0,0.0,0.0,1.0,2.0
1196,0.0,0.0,5.0,0.0,2.5,0.0,1.5,5.5,1.0,0.0,...,2.0,7.0,14.0,15.0,5.0,2.0,1.0,1.0,2.0,4.0
1197,2.0,0.0,5.0,0.0,4.0,1.5,0.5,3.0,0.5,0.0,...,6.0,9.0,12.0,11.0,6.0,1.0,0.0,1.0,0.0,1.0
1198,1.5,0.0,5.0,0.0,0.0,0.0,3.0,5.5,4.5,0.0,...,13.0,15.0,13.0,11.0,8.0,6.0,5.0,4.0,4.0,3.0


In [60]:
clf = RandomForestClassifier()
model = clf.fit(X_train, y_train)
y_pred = model.predict(X_test)
a_score = round(float(accuracy_score(y_test, y_pred)),4)
print('Accuracy', a_score)

Accuracy 0.7742


In [ ]:
# 2. TUNING
# Split combinations Grid Search with CV
clf=RandomForestClassifier()
n_estimators = {'n_estimators':[300, 500, 700, 800, 900]}; criterion = {'criterion':['gini', 'entropy']}; max_depth = {'max_depth':[5, 8, 15, 25, 30, None]}
list = [n_estimators, criterion, max_depth]
df_scores = myfns.split_grid_search(clf, X_train, y_train, X_test, y_test, list)
df_scores

Score : 0.7775 - Best Parameters : {'n_estimators': 900}
Score : 0.7542 - Best Parameters : {'criterion': 'gini'}
Score : 0.7592 - Best Parameters : {'max_depth': 30}
Score : 0.7758 - Best Parameters : {'criterion': 'entropy', 'n_estimators': 800}


In [ ]:
# 3. JCD and Tamura features for each dataset
# TRAIN
X_train = df_train.iloc[:,1:] # drop 1st (label) col
X_train = X_train.iloc[:,:186] # 168 (JCD) + 18 (Tamura) = 186
# TEST
X_test = df_test.iloc[:,1:] # drop 1st (label) col
X_test = X_test.iloc[:,:186] # 168 (JCD) + 18 (Tamura) = 186
X_test


In [57]:
clf = RandomForestClassifier()
model = clf.fit(X_train, y_train)
y_pred = model.predict(X_test)
a_score = round(float(accuracy_score(y_test, y_pred)),4)
print('Accuracy', a_score)




0.7283